In [2]:
import anndata
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import scanpy as sc
import sklearn.decomposition #import TruncatedSVD
import scipy 
import sparse, io
import os 
matplotlib.rcParams.update({'font.size': 12})
%config InlineBackend.figure_format = 'retina'

#os.chdir('/lunarc/nobackup/projects/regen_immuno/tools/samap/transcriptomes/')

In [25]:
#Load in the data
adata_pl=anndata.read_h5ad('/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Inter_data/spleen_merged_raw.h5ad')
#adata_hum=anndata.read_h5ad('/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Raw_data/Anno/progen.h5ad')
adata_ax=anndata.read_h5ad('/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Inter_data/combined_data_withobs_withleiden.h5ad')

In [4]:
adata_ax.obs

,tissue,anno2,leiden,batch
COL60_ATCCGAACGGTATCCTCAAT,Spleen,Epithelial cell,17,WT1
COL60_CAGGAATCGGCGATGGCGCC,Spleen,Epithelial cell,1,WT1
COL60_ATATCTTCCGGCCGGTCGAG,Spleen,Immune cell,13,WT1
COL60_TGCGTCTATACCAGGCCGCA,Spleen,Immune cell,13,WT1
COL60_CGGAGCGTCATTGATGCTAT,Spleen,Epithelial cell,14,WT1
...,...,...,...,...
COL21_GCGTAAGATCTCGATATTGA,Spleen,Immune cell,1,WT2
COL21_TGAGGTTAGCAACGGACCAA,Spleen,Immune cell,1,WT2
COL21_GTTCGAGGCGGGACTCATTG,Spleen,Immune cell,1,WT2
COL21_TTATTGCTAATCGTAGAGAA,Spleen,Immune cell,7,WT2


In [26]:
adata_ax

AnnData object with n_obs × n_vars = 35549 × 20278
    obs: 'tissue', 'anno2', 'leiden', 'batch'

In [51]:
#adata_hum.obs
#for i in adata_ax.var.index:
#    if i.startswith('mRNA'):
#        adata_ax.var.loc[i,'gene_ids']=i
#    else:
#        adata_ax.var.loc[i,'gene_ids']=i.split('-')[0]+'_'+i.split('-')[1]

/tmp/ipykernel_2190011/1682433122.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'AMEXTC_0340000057334' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  adata_ax.var.loc[i,'gene_ids']=i.split('-')[0]+'_'+i.split('-')[1]


In [52]:
adata_ax.var['gene_ids']

AMEXTC-0340000057334-cathepsin       AMEXTC_0340000057334
AMEXTC-0340000022263-DAPK1           AMEXTC_0340000022263
AMEXTC-0340000002427-CSF1R           AMEXTC_0340000002427
AMEXTC-0340000119775-LOC101397869    AMEXTC_0340000119775
AMEXTC-0340000236654-PHF21A          AMEXTC_0340000236654
                                             ...         
AMEXTC-0340000060502-matrix          AMEXTC_0340000060502
AMEXTC-0340000041891-type            AMEXTC_0340000041891
AMEXTC-0340000028963-LHFPL4          AMEXTC_0340000028963
AMEXTC-0340000034434-LOC103173272    AMEXTC_0340000034434
AMEXTC-0340000035626-ELFN2           AMEXTC_0340000035626
Name: gene_ids, Length: 3000, dtype: object

In [27]:
from sklearn.impute import SimpleImputer
import numpy as np

# Features with missing values
features_with_missing_values = [87, 217, 371, 445, 487, 750, 1160, 1305, 1590, 1704,
                                1751, 1766, 1945, 1959, 2560, 2573, 2634, 2911, 2977, 3259, 3379,
                                3571, 3612, 3818, 4190, 4221, 4428, 4615, 4619, 4708, 4729, 4801,
                                4865, 4896, 4958, 5006, 5055, 5094, 5247, 5399, 5786, 6014, 6479,
                                6629, 7416, 7584, 7643, 8894, 9107, 9302, 9349, 9563, 9646, 9743,
                                9773, 9794, 9804, 10156, 10253, 10347, 10368, 10370, 10613, 10691,
                                10749, 11086, 11343, 11560, 12174, 12401, 12617, 13081, 13112,
                                13157, 14370, 14781, 15348, 15551, 15584, 15629, 15749, 16112,
                                16728, 16788, 17220, 17660, 17766, 18238, 18646, 19731, 19948,
                                20056, 20182, 20263]
# Remove features with missing values
raw_counts_data = np.delete(adata_ax.X, features_with_missing_values, axis=1)

# Replace inf values with a large finite value
raw_counts_data[raw_counts_data == np.inf] = 1e9

# Create a SimpleImputer instance with the mean strategy
imputer = SimpleImputer(strategy='mean')

# Fit the imputer on the data and transform it
raw_counts_data_imputed = imputer.fit_transform(raw_counts_data)

if raw_counts_data_imputed.shape[1] != adata_ax.n_vars:
    # Pad with zeros to match the original number of features
    raw_counts_data_imputed = np.pad(raw_counts_data_imputed, ((0, 0), (0, adata_ax.n_vars - raw_counts_data_imputed.shape[1])), mode='constant', constant_values=0)

# Create a new AnnData object with the imputed data
adata_ax_raw = adata_ax.copy()
adata_ax_raw.X = raw_counts_data_imputed

# Create a new AnnData object with the imputed data
adata_ax_raw = adata_ax.copy()


adata_ax_raw.X = raw_counts_data_imputed


In [33]:
adata_ax.var_names

Index(['AMEXTC_0340000000001_HHIP', 'AMEXTC_0340000000004_PACRGL',
       'AMEXTC_0340000000005_ANGPT4', 'AMEXTC_0340000000006_cnot3',
       'AMEXTC_0340000000009_scin', 'AMEXTC_0340000000012_TMEM47',
       'AMEXTC_0340000000013_MGC145330', 'AMEXTC_0340000000014_LOC101944747',
       'AMEXTC_0340000000020_SYNPO2L', 'AMEXTC_0340000000026_KCND3',
       ...
       'mRNA01381', 'mRNA01387', 'mRNA01388', 'mRNA01391', 'mRNA01392',
       'mRNA01394', 'mRNA01395', 'mRNA01396', 'mRNA01397', 'mRNA01398'],
      dtype='object', length=20278)

In [28]:
#save the raw data
adata_ax_raw.write_h5ad('/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Inter_data/adata_ax_raw_imputed.h5ad')

In [65]:
adata_ax_raw.X

array([[-0.27841458, -0.12192587, -0.18591334, ...,  0.        ,
         0.        ,  0.        ],
       [-0.28823042, -0.12856661, -0.19308153, ...,  0.        ,
         0.        ,  0.        ],
       [-0.22299844, -0.09050886, -0.15187292, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.21590479, -0.08694632, -0.14802304, ...,  0.        ,
         0.        ,  0.        ],
       [-0.2207708 , -0.08938226, -0.15065499, ...,  0.        ,
         0.        ,  0.        ],
       [-0.25875258, -0.10984658, -0.17282453, ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [7]:
#Samap analysis
from samap.mapping import SAMAP
from samap.analysis import (get_mapping_scores, GenePairFinder,
                            sankey_plot, chord_plot, CellTypeTriangles,
                            ParalogSubstitutions, FunctionalEnrichment,
                            convert_eggnog_to_homologs, GeneTriangles)

/home/nikvaku/PycharmProjects/samap/lib/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


In [29]:
org1='/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Inter_data/spleen_merged_raw.h5ad'
#org2='/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Raw_data/Anno/progen.h5ad'
org2="/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Inter_data/adata_ax_raw_imputed.h5ad"
filenames = {'pw':org1,'ax':org2}
sm = SAMAP(
        filenames,
        f_maps = '/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Inter_data/maps/',
        save_processed=True, #if False, do not save the processed results to `*_pr.h5ad`
        keys = {'pw':'leiden','ax':'leiden'},
   )
sm.run()
samap = sm.samap

Processing data pw from:
/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Inter_data/spleen_merged_raw.h5ad
RUNNING SAM
Iteration: 0, Convergence: 1.0
Iteration: 1, Convergence: 0.8635069960641922
Computing the UMAP embedding...


/home/nikvaku/.local/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Elapsed time: 63.29446053504944 seconds
Not updating the manifold...
Processing data ax from:
/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Inter_data/adata_ax_raw_imputed.h5ad
RUNNING SAM
Iteration: 0, Convergence: 1.0
Iteration: 1, Convergence: 0.9053773472272745
Iteration: 2, Convergence: 0.013809481344739334
Computing the UMAP embedding...


/home/nikvaku/.local/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Elapsed time: 58.12323451042175 seconds
Not updating the manifold...
0 `pw` gene symbols match between the datasets and the BLAST graph.
0 `ax` gene symbols match between the datasets and the BLAST graph.


ValueError: zero-size array to reduction operation

In [39]:
keys_ct = {'pw':'leiden','hs':'cell_type'}
Ct,MappingTable_ct = get_mapping_scores(sm,keys_ct,n_top = 100)

In [40]:
MappingTable_ct

,pw_0,pw_1,pw_10,pw_11,pw_12,pw_13,pw_14,pw_15,pw_2,pw_3,...,hs_early lymphoid progenitor,hs_fraction A pre-pro B cell,hs_granulocyte monocyte progenitor cell,hs_hematopoietic multipotent progenitor cell,hs_hematopoietic stem cell,hs_megakaryocyte,hs_megakaryocyte-erythroid progenitor cell,hs_native cell,hs_promonocyte,hs_promyelocyte
pw_0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.858578,0.907846,0.524759,0.812766,0.793911,0.533153,0.659531,0.475465,0.327531,0.454378
pw_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.461382,0.808084,0.864115,0.970783,0.963101,0.634761,0.958459,0.403769,0.088351,0.033038
pw_10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.016967,0.145901,0.216924,0.049950,0.091438,0.222700,0.365765,0.029219,0.212445,0.107861
pw_11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.007445,0.143575,0.216239,0.164880,0.147721,0.199405,0.324919,0.033563,0.188873,0.157241
pw_12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.016645,0.030631,0.007256,0.024705,0.012777,0.008045,0.005255,0.024855,0.005982,0.006523
pw_13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001661,0.000000,0.589968,0.000420,0.001025,0.011461,0.000189,0.001461,0.609975,0.479638
pw_14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010189,0.001989,0.000000,0.001994,0.000594,0.002481,0.006617,0.015793,0.000000,0.000000
pw_15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.005887,0.005606,0.000435,0.003433,0.049738,0.017942,0.095043,0.005651,0.000000,0.000000
pw_2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.087245,0.099935,0.419976,0.167991,0.050220,0.563259,0.455568,0.133419,0.036809,0.149087
pw_3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.005739,0.004536,0.120140,0.434506,0.140833,0.891751,0.533274,0.104491,0.397493,0.053790


In [41]:
MappingTable_ct.to_csv("/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Inter_data/Integrated_SAMAP_table_progen.csv")

In [34]:
pd.read_csv("/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Inter_data/Integrated_SAMAP_table_bonemarrow.csv")

,Unnamed: 0,pw_0,pw_1,pw_10,pw_11,pw_12,pw_13,pw_14,pw_15,pw_2,...,hs_natural killer cell,hs_non-classical monocyte,hs_non-terminally differentiated cell,hs_plasma cell,hs_plasmacytoid dendritic cell,hs_pre-natural killer cell,hs_precursor B cell,hs_pro-B cell,hs_small pre-B-II cell,hs_unswitched memory B cell
0,pw_0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.023662,0.436827,0.0,0.160018,0.033033,0.025120,0.319201,0.244634,0.026738,0.598933
1,pw_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.005354,0.017144,0.0,0.416959,0.019944,0.001040,0.103135,0.236459,0.024535,0.626013
2,pw_10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000157,0.037931,0.0,0.039893,0.000999,0.000000,0.000311,0.000000,0.000000,0.153589
3,pw_11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000193,0.018497,0.0,0.011948,0.000835,0.000000,0.005909,0.003764,0.000000,0.128945
4,pw_12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.068464,0.002349,0.0,0.007508,0.000248,0.000000,0.000202,0.000065,0.000000,0.012936
5,pw_13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000497,0.001026,0.0,0.000000,0.349006,0.000000,0.000000,0.000000,0.000000,0.001327
6,pw_14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.079279,0.005734,0.0,0.001081,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,pw_15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007900
8,pw_2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.061693,0.667903,0.0,0.021265,0.190399,0.000240,0.000794,0.000485,0.000000,0.015209
9,pw_3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.089586,0.052986,0.0,0.009082,0.001842,0.000797,0.002363,0.000732,0.000000,0.013468
